# **Rag System using Llama2**

In [8]:
!pip install pypdf

In [9]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [10]:
#Embedding
!pip install sentence_transformers

In [11]:
%pip install llama-index-llms-huggingface
!pip install llama-index


In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [13]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='c72b6e90-f019-4ed9-8964-1b50ef8ddf3d', embedding=None, metadata={'page_label': '1', 'file_name': 'Abuse_Handling.pdf', 'file_path': '/content/data/Abuse_Handling.pdf', 'file_type': 'application/pdf', 'file_size': 53687, 'creation_date': '2025-09-04', 'last_modified_date': '2025-09-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Abuse Handling \n \n## Section 3.1 – Abuse Categories \n- Phishing websites \n- Malware distribution \n- Spam and botnet control \n \n## Section 3.2 – Escalation \n- All abuse-related tickets must be escalated to the Abuse Team. \n- Escalation email: abuse@company.example \n \n## Section 3.3 –

In [14]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Answer the question in the following format:
  "answer",
  "references",
  "action_required":
"""

#Deafult Format Supported by LLAMA2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")


Here we connect the HuggingFace CLI(command line interface) with our pipeline. A token is generated using a huggingface account and is added as a git credential.

In [15]:
!!huggingface-cli login

["\x1b⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.\x1b",
 '',
 '    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|',
 '    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|',
 '    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|',
 '    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|',
 '    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|',
 '',
 '    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .',
 'Enter your token (input will not be visible): ',
 'Add token as git credential? (Y/n) Y',
 'Token is valid (permission: write).',
 'The token `Tucows Interview` has been saved to /root/.cache/hu

For the following to work, make sure that proper permissions are requested from Meta on the HuggingFace website: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

Alternatively, if permissions are not granted use:
"mistralai/Mistral-7B-Instruct-v0.3". It is free and requires the user to agree to its terms and user rights are granted immediately.

In [16]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True} #We load it in 8bit since quantization takes place
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [17]:
!pip install -U langchain-community
!pip install llama-index-embeddings-langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from llama_index.core.service_context import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

/tmp/ipython-input-2083404176.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Now we have the Embedded Model, LLM model and the documents. So we will combine them all using the settings.

In [19]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [20]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

In [21]:
index

In [22]:
query_engine = index.as_query_engine(llm=llm)

In [23]:
response = query_engine.query("What are the different abuse categories and who should such tickets be escalated to?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [24]:
print (response)


The different abuse categories mentioned in the provided context are:

* Phishing websites
* Malware distribution
* Spam and botnet control

Tickets related to these categories should be escalated to the Abuse Team. The escalation email for abuse-related tickets is mentioned in Section 3.2 of the Abuse Handling document: abuse@company.example.

References:

* Abuse Handling document (Section 3.1 and 3.2)
* Domain Suspension Guidelines document (Section 1.1 and 1.2)

Action Required: None.


After getting the required response, the response is exported to a JSON output file which is stored in the same folder as the documents (/content/data)

In [34]:
from pathlib import Path
import json
from datetime import datetime

def save_llm_response(llama_response, pdfs_dir: str = "/content/data", filename: str | None = None) -> Path:
    """
    Save data from a LlamaIndex Response object as JSON in the specified directory.
    Returns the path to the saved JSON file.
    """
    pdfs_path = Path(pdfs_dir)
    pdfs_path.mkdir(parents=True, exist_ok=True)

    # Extract data from LlamaIndex Response object
    data = {
        "answer": str(llama_response),  # The main generated response text
        "references": [],
        "action_required": "" # The action required part seems to be generated by the LLM based on the prompt, which is now part of the 'answer' string. We can try to parse it out if needed, but for now, let's keep it simple.
    }

    # Extract references from source nodes
    if hasattr(llama_response, "source_nodes"):
        references = []
        for node in llama_response.source_nodes:
            metadata = node.metadata
            file_name = metadata.get("file_name", "N/A")
            page_label = metadata.get("page_label", "N/A")
            references.append(f"{file_name} (Page {page_label})")
        data["references"] = references

    # Attempt to extract "action_required" from the answer string using regex, as it seems to be a custom part of the prompt
    action_required_match = re.search(r"Action Required:\s*(.*)", data["answer"], re.DOTALL | re.IGNORECASE)
    if action_required_match:
        data["action_required"] = action_required_match.group(1).strip()
        # Remove the "Action Required" part from the main answer
        data["answer"] = re.sub(r"Action Required:\s*(.*)", "", data["answer"], flags=re.DOTALL | re.IGNORECASE).strip()


    # Choose a filename
    if not filename:
        ts = datetime.now().strftime("%Y%m%d-%H%M%S")
        filename = f"llm_response_{ts}.json"

    out_path = pdfs_path / filename
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    return out_path

# usage:
# Pass the LlamaIndex Response object directly to the function.
saved_path = save_llm_response(response, pdfs_dir="/content/data")
print(f"Saved JSON to: {saved_path}")

Saved JSON to: /content/data/llm_response_20250904-025607.json
